## Read .ghg metadata

In [ ]:
import os
import time
import pandas as pd
from Parse_GHG import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

# Site = 'BB2'

# Values we expect to change
dynamicMetaData_Headers = ['altitude','canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders

N_recs = 4#8*31
def find_new_GHG(Site,make_new=False):

    i = 0
    Dpath = 'C:\\highfreq\\'
    raw_dir = Dpath+Site+'\\raw\\'
    meta_dir = Dpath+Site+'\\metadata\\'
    meta_file = meta_dir+'GHGMetaData.csv'

    if os.path.isfile(meta_file) and make_new is False:
        Records = pd.read_csv(meta_file,parse_dates=['TimeStamp'])
        FileNames = Records['filename'].tolist()
    else:
        Records = pd.DataFrame()
        FileNames = []
        if not os.path.exists(meta_dir):
            os.mkdir(meta_dir)
    

    for (root, dirs, files) in sorted(os.walk(raw_dir)):
        # if root.__contains__('raw') :
        for file in files:
            name, tag = file.split('.')
            # .ghg files are located at the end of each directory tree
            # Avoids reading any that might be misplaced elsewhere
            if tag == 'ghg' and len(dirs)==0 and file not in FileNames:  
                
                if i <= 48*5:
                    GHG_out = read_GHG(root,name)
                    Records = pd.concat(
                        [Records,GHG_out.Summary],
                    axis=0,
                    ignore_index=True
                    )
                    i += 1
            f.value = i/len(files)
    return(Records,meta_dir,i)

Records,meta_dir,i = find_new_GHG('BB')

Records.to_csv(meta_dir+'GHGMetaData.csv',index=False)

elapsed = time.time()-T1
print('Processing time ', elapsed) 
print('To inspect ', i, ' .ghg files')
print('Seconds/file ', elapsed/(i+1)) 

staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
print(staticMetaData)

dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
dynamicMetaData['date']=dynamicMetaData.index.date
dynamicMetaData['time']=dynamicMetaData.index.time
dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
dynamicMetaData

# Processing time  65.87000775337219
# To inspect  241  .ghg files

Records['altitude'].astype(float).describe()


In [40]:
import matlab.engine
eng = matlab.engine.start_matlab()
# eng.triarea(nargout=0)

In [43]:
eng.runEddyPro(['02-Jul-2022','03-Jul-2022'],'BB','C:\highfreq','C:\temp\EP_Testing\EP_Output_Test',0,'12:00','13:00',[],2,nargout=0)

In [59]:
# eng.datetime(2022,7,1):
# matlab.
# from datetime import datetime
# import pandas as pd
# pd.date_range().date.tolist()

import configparser
config = configparser.ConfigParser()

config.read_file(open('C:/highfreq/BB/EP_templates/BB_Template.eddypro'))

for k in config.keys():
    print(k)
    print()
    for j in config[k].keys():
        print(j, config[k][j])




DEFAULT

Project

creation_date 2020-06-09T16:08:12
last_change_date 2023-02-15T17:44:32
file_name C:/highfreq/BB/EP_templates/BB_template.eddypro
run_mode 0
run_fcc 0
project_title processing
sw_version 7.0.9
ini_version 4.5.0
project_id BBTest
file_type 0
file_prototype yyyy-mm-ddTHHMM??_LI-7200.ghg
use_pfile 0
proj_file 
use_dyn_md_file 0
dyn_metadata_file 
binary_hnlines -1
binary_eol -1
binary_nbytes -1
binary_little_end -1
master_sonic csat3_1
col_co2 26
col_h2o 28
col_ch4 50
col_n2o 0
col_int_t_1 31
col_int_t_2 32
col_int_p 15
col_air_t 51
col_air_p 52
col_cell_t 30
col_diag_75 0
col_diag_72 4
col_diag_77 54
col_diag_anem 0
gas_mw -1.0000
gas_diff -1.00000
col_ts 0
out_rich 1
out_metadata 0
make_dataset 0
pr_subset 1
pr_start_date 2022-07-01
pr_start_time 12:00
pr_end_date 2022-07-01
pr_end_time 13:00
hf_meth 1
lf_meth 1
wpl_meth 1
foot_meth 1
tob1_format 0
out_path C:/highfreq/BB/EP_outputs
fix_out_format 1
err_label NaN
qc_meth 1
use_biom 0
biom_file 
biom_dir 
biom_rec 0
biom